## Define LLM and Embedding model

In [1]:
# !pip install llama-index

In [ ]:
# pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

In [ ]:
# CMAKE_ARGS="-DLLAMA_CUDA=on"

In [ ]:
# pip install llama-cpp-python

In [ ]:
# %pip install llama-index-embeddings-huggingface
# %pip install llama-index-llms-llama-cpp

In [2]:
# pip install -U langsmith

In [2]:
import torch
torch.cuda.is_available()

True

In [3]:
import nest_asyncio

nest_asyncio.apply()

In [12]:
import os
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = "llamaIndex_HCC"
os.environ["LANGCHAIN_API_KEY"] = "LANGCHAIN_API_KEY"

In [13]:
from langsmith import traceable

In [1]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings

embed_model = HuggingFaceEmbedding(
    model_name="maiduchuy321/vietnamese-bi-encoder-fine-tuning-for-law-chatbot")



C:\Users\maidu\anaconda3\envs\LamaIndex\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
Settings.embed_model = embed_model

In [6]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.llms.llama_cpp import LlamaCPP
from llama_index.llms.llama_cpp.llama_utils import (
    messages_to_prompt,
    completion_to_prompt,
)

In [24]:
llm = LlamaCPP(
    # You can pass in the URL to a GGML model to download it automatically
    model_url="https://huggingface.co/uonlp/Vistral-7B-Chat-gguf/resolve/main/ggml-vistral-7B-chat-q4_0.gguf?download=true"
    ,
    # optionally, you can set the path to a pre-downloaded model instead of model_url
    # model_path="model_path",
    temperature=0.1,
    max_new_tokens=256,
    # llama2 has a context window of 4096 tokens, but we set it lower to allow for some wiggle room
    context_window=2048,
    # kwargs to pass to __call__()
    generate_kwargs={},
    # kwargs to pass to __init__()
    # set to at least 1 to use GPU
    model_kwargs={"n_gpu_layers": 30},
    # transform inputs into Llama2 format
    # messages_to_prompt=messages_to_prompt,
    # completion_to_prompt=completion_to_prompt,
    verbose=True,
)

llama_model_loader: loaded meta data with 22 key-value pairs and 291 tensors from I:/Hoc/do an/model/ggml-vistral-7B-chat-q4_1.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = .
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.head_

In [25]:
Settings.llm = llm

In [ ]:
response = llm.complete("Năm nay là năm bao nhiêu")
print(response.text)

## Ingestion

### Load data

In [3]:
import pandas as pd

df = pd.read_csv("procedureHGG2.xlsx")

In [4]:
df

,_id,code,level.name.0.name,agencyLevel.0.name.0.name,sector.name.0.name,translate.0.name,translate.0.processingTime,translate.0.results,translate.0.requirement,translate.0.legalGrounds,translate.0.steps,translate.0.implementationMethod,translate.0.implementer,translate.0.implementMethod,translate.0.dossierComponent,translate.0.fee,translate.0.serviceFee,translate.0.agencyAccept,translate.0.agencyCombination,status
0,644a9a7e70fb541ddeb9f69e,2.000379.000.00.00.H22,TTHC chưa cung cấp DVCTT,Cấp Huyện,Đất đai,Bán hoặc góp vốn bằng tài sản gắn liền với đất...,Trực tiếp: 30 Ngày,Giấy chứng nhận quyền sử dụng đất quyền sở hữu...,- Yêu cầu đối với bên bán tài sản gắn liền với...,Luật 45/2013/QH13\nThông tư 02/2014/TT-BTC\nNg...,"(1) Người sử dụng đất thực hiện mua bán, góp v...",Trực tiếp,Công dân Việt Nam,1\nDo Ủy ban nhân dân cấp tỉnh quy định nhưng ...,"- Hợp đồng, văn bản mua bán, góp vốn bằng tài ...",Trực tiếp:,0,Văn phòng đăng ký quyền sử dụng đất cấp huyện,0,1
1,644a9a7e70fb541ddeb9f6a4,1.001991.000.00.00.H22,TTHC chưa cung cấp DVCTT,Tỉnh/ Thành phố,Đất đai,Bán hoặc góp vốn bằng tài sản gắn liền với đất...,Trực tiếp: 30 Ngày,Giấy chứng nhận quyền sử dụng đất quyền sở hữu...,(1) Yêu cầu đối với bên bán tài sản gắn liền v...,Luật 45/2013/QH13\nThông tư 02/2014/TT-BTC\nNg...,"(1) Người sử dụng đất thực hiện mua bán, góp v...",Trực tiếp,"Công dân Việt Nam, Người Việt Nam định cư ở nư...",0,"Hợp đồng, văn bản mua bán, góp vốn bằng tài sả...",Trực tiếp:,0,Văn phòng đăng ký đất đai\nChi nhánh Văn phòng...,0,1
2,644a9a7f70fb541ddeb9f6a8,1.002109.000.00.00.H22,TTHC chưa cung cấp DVCTT,Cấp Tỉnh,Đất đai,Bán hoặc góp vốn bằng tài sản gắn liền với đất...,Trực tiếp: 30 Ngày,Giấy chứng nhận quyền sử dụng đất quyền sở hữu...,(1) Yêu cầu đối với bên bán tài sản gắn liền v...,Thông tư 02/2014/TT-BTC\nNghị định 43/2014/NĐ-...,(1) Người sử dụng đất nộp hồ sơ tại Văn phòng ...,Trực tiếp,"Công dân Việt Nam, Người Việt Nam định cư ở nư...",1\nDo Ủy ban nhân dân cấp tỉnh quy định nhưng ...,"Hợp đồng, văn bản mua bán, góp vốn bằng tài sả...",Trực tiếp:,0,Văn phòng đăng ký quyền sử dụng đất cấp tỉnh,0,1
3,644a9a7f70fb541ddeb9f6ad,1.007767.000.00.00.H22,TTHC chưa cung cấp DVCTT,Cấp Tỉnh,Nhà ở và công sở,Bán nhà ở cũ thuộc sở hữu nhà nước,Trực tiếp: 45 Ngày,Quyết định đối tượng được mua nhà ở cũ thuộc s...,"a) Yêu cầu, điều kiện 1:\n- Người đề nghị mua ...",Quy định chi tiết và hướng dẫn thi hành một số...,-Bước 1: - Người mua nhà ở phải nộp hồ sơ đề n...,Trực tiếp,"Công dân Việt Nam, Người nước ngoài",1\nkhông quá 45 ngày kể từ ngày đơn vị quản lý...,- Đơn đề nghị mua nhà ở cũ;\n- Hợp đồng thuê n...,Trực tiếp:,0,Đơn vị quản lý vận hành nhà ở,0,1
4,644a9a8070fb541ddeb9f6b3,1.000105.000.00.00.H22,Toàn trình,Cơ Quan Hành Chính,Việc làm,"Báo cáo giải trình nhu cầu, thay đổi nhu cầu s...",Trực tiếp: 10 Ngày làm việc\nTrực tuyến: 10 Ng...,Chấp thuận vị trí công việc sử dụng người lao ...,Người sử dụng lao động xác định được nhu cầu s...,QUY ĐỊNH VỀ NGƯỜI LAO ĐỘNG NƯỚC NGOÀI LÀM VIỆC...,- Bước 1: Trước ít nhất 30 ngày kể từ ngày dự ...,"Trực tiếp, Trực tuyến, Dịch vụ bưu chính","Doanh nghiệp, Tổ chức (không bao gồm doanh ngh...",0,\nBáo cáo giải trình nhu cầu sử dụng người lao...,Trực tiếp: 0\nTrực tuyến: 0\nDịch vụ bưu chính...,0,Ủy ban nhân dân cấp Tỉnh Sở Lao động - Thương ...,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1941,64b79c4ce24db3208b71b21d,1.004583,TTHC chưa cung cấp DVCTT,Sở,Đăng ký biện pháp bảo đảm,"Đăng ký thế chấp quyền sử dụng đất, tài sản gắ...",:\nTrong ngày nhận hồ sơ hợp lệ. Nếu nhận hồ s...,0,"Cá nhân, hộ gia đình, pháp nhân.",1.11. Căn cứ pháp lý của thủ tục hành chính:\n...,1.1. Trình tự thực hiện:\na) Nộp hồ sơ:\n- Văn...,"1. Đăng ký thế chấp quyền sử dụng đất, tài sản...",0,0,Mẫu số 01/ĐKTC - Phiếu yêu cầu đăng ký thế chấ...,gắn liền với đất đối với trường hợp hồ sơ có G...,Hội đồng nhân dân cấp tỉnh quyết định đối với ...,a) Nộp hồ sơ:\n- Văn phòng đăng ký đất đai.\n\...,0,1
1942,64b81a0

#### Metadata gồm có: 
- Mã TTHC 
- Trạng thái thực hiện 
- Cấp độ thực hiện
- Lĩnh vực
- Tên thủ tục
- Thời gian trả kết quả
- Yêu cầu điều kiện
- Căn cứ pháp lý
- Trình tự thực hiện
- Cách thức thực hiện
- Đối tượng thực hiện
- Thời gian thực hiện
- Thành phần hồ sơ
- Lệ phí
- Phí dịch vụ
- Cơ quan giải quyết thủ tục hành chính
- Cơ quan phối hợp
- status

In [5]:
list_strings = []

for index, row in df.iterrows():
    row_string = ', '.join([
        f'Tên thủ tục: {row["translate.0.name"]}',
        f'Yêu cầu điều kiện: {row["translate.0.requirement"]}',
        f'Tên kết quả TTHC: {row["translate.0.results"]}',
        f'Trình tự thực hiện: {row["translate.0.steps"]}',
        f'Cách thức thực hiện: {row["translate.0.implementMethod"]}',
        f'Thành phần hồ sơ: {row["translate.0.dossierComponent"]}'
    ])
    list_strings.append(row_string)


In [12]:
from llama_index.core import Document

documents_dachuanhoa = [
    Document(
        text= ', '.join([
        f'Tên thủ tục: {row["translate.0.name"]}',
        f'Yêu cầu điều kiện: {row["translate.0.requirement"]}',
        f'Tên kết quả TTHC: {row["translate.0.results"]}',
        f'Trình tự thực hiện: {row["translate.0.steps"]}',
        f'Cách thức thực hiện: {row["translate.0.implementMethod"]}',
        f'Thành phần hồ sơ: {row["translate.0.dossierComponent"]}'
    ]),
        metadata={
            'code': row['code'],
            'implementationStatus': row['level.name.0.name'],
            'agencyLevel': row['agencyLevel.0.name.0.name'],
            'sector': row['sector.name.0.name'],
            'processingTime': row['translate.0.processingTime'],
            'legalGrounds':	row['translate.0.legalGrounds'], ###new
            'implementationMethod': row['translate.0.implementationMethod'],
            'implementer': row['translate.0.implementer'],
            'fee': row['translate.0.fee'],
            'serviceFee': row['translate.0.serviceFee'],
            'agencyAccept': row['translate.0.agencyAccept'],
            'agencyCombination': row['translate.0.agencyCombination'],
            'status': row['status'],
        }
    )
    for _, row in df.iterrows() 
]

In [13]:
len(documents_dachuanhoa)

1946

### Vector Stores

#### Creating a Weaviate Client

In [ ]:
# %pip install llama-index-vector-stores-weaviate

In [ ]:
# pip install -U weaviate-client

In [17]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [18]:
import weaviate

In [ ]:
# import weaviate
# import os
  
# Connect to a WCS instance
# client = weaviate.connect_to_wcs(
#     cluster_url="cluster_url",
#     auth_credentials=weaviate.auth.AuthApiKey("API_KEY"))


In [19]:
# local
client = weaviate.connect_to_local(
)

INFO:httpx:HTTP Request: GET http://localhost:8080/v1/.well-known/openid-configuration "HTTP/1.1 404 Not Found"
HTTP Request: GET http://localhost:8080/v1/.well-known/openid-configuration "HTTP/1.1 404 Not Found"
INFO:httpx:HTTP Request: GET http://localhost:8080/v1/meta "HTTP/1.1 200 OK"
HTTP Request: GET http://localhost:8080/v1/meta "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://pypi.org/pypi/weaviate-client/json "HTTP/1.1 200 OK"
HTTP Request: GET https://pypi.org/pypi/weaviate-client/json "HTTP/1.1 200 OK"


In [20]:
print(client.is_ready())

INFO:httpx:HTTP Request: GET http://localhost:8080/v1/.well-known/ready "HTTP/1.1 200 OK"
HTTP Request: GET http://localhost:8080/v1/.well-known/ready "HTTP/1.1 200 OK"
True


### Split data

In [21]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.vector_stores.weaviate import WeaviateVectorStore
from IPython.display import Markdown, display

#### Define Semantic Splitter

In [22]:
from llama_index.core.node_parser import (
    SemanticSplitterNodeParser,
)

In [27]:
from langsmith import traceable
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
@traceable
def load_embeddings(model_name):
    embeddings = HuggingFaceEmbedding(
        model_name=model_name,device="cuda",embed_batch_size=5,trust_remote_code=True
    )
    return embeddings

In [23]:
splitter = SemanticSplitterNodeParser(
    buffer_size=1, breakpoint_percentile_threshold=95, embed_model=embed_model
)

In [24]:
nodesSemanticSplitter = splitter.get_nodes_from_documents(documents_dachuanhoa)

Batches: 100%|██████████| 1/1 [00:00<00:00, 26.07it/s]


In [28]:
len(nodesSemanticSplitter)

4009

In [33]:
from llama_index.core import StorageContext

In [35]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings

embed_model_weaviate = HuggingFaceEmbedding(
    model_name="maiduchuy321/vietnamese-bi-encoder-fine-tuning-for-law-chatbot",device="cuda",embed_batch_size=32
)

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: I:/Hoc/Public Administration/vietnamese-bi-encoder-fine-tuning-for-law-chatbot_epoch15
Load pretrained SentenceTransformer: I:/Hoc/Public Administration/vietnamese-bi-encoder-fine-tuning-for-law-chatbot_epoch15
INFO:sentence_transformers.SentenceTransformer:2 prompts are loaded, with the keys: ['query', 'text']
2 prompts are loaded, with the keys: ['query', 'text']


In [36]:
client.collections.delete("HCC_with_Vietnamese_Bi_Encoder_Fine_Tuning_For_Law_Chatbot")  # Replace with your collection name

vector_store_HCC = WeaviateVectorStore(
    weaviate_client=client, index_name="HCC_with_Vietnamese_Bi_Encoder_Fine_Tuning_For_Law_Chatbot", tenant="procedureHGG2"
)
storage_context_HCC = StorageContext.from_defaults(vector_store=vector_store_HCC)

index_HCC = VectorStoreIndex(
    nodesSemanticSplitter, embed_model=embed_model_weaviate, storage_context=storage_context_HCC
)

INFO:httpx:HTTP Request: DELETE http://localhost:8080/v1/schema/HCC_with_Vietnamese_Bi_Encoder_Fine_Tuning_For_Law_Chatbot "HTTP/1.1 200 OK"
HTTP Request: DELETE http://localhost:8080/v1/schema/HCC_with_Vietnamese_Bi_Encoder_Fine_Tuning_For_Law_Chatbot "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://localhost:8080/v1/schema/HCC_with_Vietnamese_Bi_Encoder_Fine_Tuning_For_Law_Chatbot "HTTP/1.1 404 Not Found"
HTTP Request: GET http://localhost:8080/v1/schema/HCC_with_Vietnamese_Bi_Encoder_Fine_Tuning_For_Law_Chatbot "HTTP/1.1 404 Not Found"
INFO:httpx:HTTP Request: POST http://localhost:8080/v1/schema "HTTP/1.1 200 OK"
HTTP Request: POST http://localhost:8080/v1/schema "HTTP/1.1 200 OK"


Batches: 100%|██████████| 1/1 [00:00<00:00,  3.90it/s]


INFO:httpx:HTTP Request: GET http://localhost:8080/v1/schema "HTTP/1.1 200 OK"
HTTP Request: GET http://localhost:8080/v1/schema "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://localhost:8080/v1/nodes "HTTP/1.1 200 OK"
HTTP Request: GET http://localhost:8080/v1/nodes "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://localhost:8080/v1/nodes "HTTP/1.1 200 OK"
HTTP Request: GET http://localhost:8080/v1/nodes "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://localhost:8080/v1/nodes "HTTP/1.1 200 OK"
HTTP Request: GET http://localhost:8080/v1/nodes "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://localhost:8080/v1/nodes "HTTP/1.1 200 OK"
HTTP Request: GET http://localhost:8080/v1/nodes "HTTP/1.1 200 OK"


Batches: 100%|██████████| 1/1 [00:00<00:00, 11.39it/s]


INFO:httpx:HTTP Request: GET http://localhost:8080/v1/nodes "HTTP/1.1 200 OK"
HTTP Request: GET http://localhost:8080/v1/nodes "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://localhost:8080/v1/nodes "HTTP/1.1 200 OK"
HTTP Request: GET http://localhost:8080/v1/nodes "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://localhost:8080/v1/nodes "HTTP/1.1 200 OK"
HTTP Request: GET http://localhost:8080/v1/nodes "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://localhost:8080/v1/nodes "HTTP/1.1 200 OK"
HTTP Request: GET http://localhost:8080/v1/nodes "HTTP/1.1 200 OK"


### Loading the index

In [37]:
from llama_index.core import VectorStoreIndex
from llama_index.vector_stores.weaviate import WeaviateVectorStore
from IPython.display import Markdown, display

vector_store = WeaviateVectorStore(
    weaviate_client=client, index_name="HCC_with_Vietnamese_Bi_Encoder_Fine_Tuning_For_Law_Chatbot"
)

loaded_index = VectorStoreIndex.from_vector_store(vector_store)

INFO:httpx:HTTP Request: GET http://localhost:8080/v1/schema/HCC_with_Vietnamese_Bi_Encoder_Fine_Tuning_For_Law_Chatbot "HTTP/1.1 200 OK"
HTTP Request: GET http://localhost:8080/v1/schema/HCC_with_Vietnamese_Bi_Encoder_Fine_Tuning_For_Law_Chatbot "HTTP/1.1 200 OK"


In [39]:
query = "Bán nhà ở cũ thuộc sở hữu nhà nước"

In [40]:
retriever = loaded_index.as_retriever()

In [41]:
nodes = retriever.retrieve(query)

Batches: 100%|██████████| 1/1 [00:00<00:00, 12.35it/s]

INFO:httpx:HTTP Request: GET http://localhost:8080/v1/schema/HCC_with_Vietnamese_Bi_Encoder_Fine_Tuning_For_Law_Chatbot "HTTP/1.1 200 OK"
HTTP Request: GET http://localhost:8080/v1/schema/HCC_with_Vietnamese_Bi_Encoder_Fine_Tuning_For_Law_Chatbot "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://localhost:8080/v1/schema/HCC_with_Vietnamese_Bi_Encoder_Fine_Tuning_For_Law_Chatbot "HTTP/1.1 200 OK"
HTTP Request: GET http://localhost:8080/v1/schema/HCC_with_Vietnamese_Bi_Encoder_Fine_Tuning_For_Law_Chatbot "HTTP/1.1 200 OK"


In [53]:
nodes

[NodeWithScore(node=TextNode(id_='05cfca71-7603-444c-acd2-d83ad76f7cca', embedding=[0.009111807681620121, 0.010679693892598152, 0.005409508012235165, -0.058139268308877945, -0.006122905761003494, -0.011487969197332859, 0.08123829960823059, -0.10772205889225006, 0.0019893611315637827, -0.014141757972538471, 0.05027279630303383, 0.004594883881509304, 0.07105112820863724, -0.0007404551142826676, -0.008597610518336296, 0.02561265043914318, -0.040976475924253464, 0.00857139565050602, 0.009251688607037067, -0.02089412324130535, -0.016542864963412285, -0.01595907472074032, 0.013316929340362549, -0.027885278686881065, 0.047343309968709946, 0.006860586814582348, -0.05185515806078911, -0.020898327231407166, -0.02096892148256302, 0.03519187867641449, -0.02051878720521927, 0.026988528668880463, 0.02617575228214264, -0.02190108224749565, 0.10146568715572357, 0.0190921388566494, -0.006731083150953054, -0.04697244241833687, -0.018923427909612656, -0.015705738216638565, 0.0026816774625331163, -0.02322

In [52]:
metadata = [item.metadata for item in nodes]
metadata[0]['legalGrounds']

'Quy định chi tiết và hướng dẫn thi hành một số điều của Luật Nhà ở\nLuật 65/2014/QH13'

In [43]:
texts = [item.text for item in nodes]
texts

In [28]:
# set Logging to DEBUG for more detailed outputs
query_engine = loaded_index.as_query_engine(
    vector_store_query_mode="hybrid",
    similarity_top_k=3,
    alpha=0.75 #very similar to vector search
)